# TODO:

* Add vertical line for means in bar/distribution plots? Or lower and higher quantiles?
* Evaluate AUROC score without setting a threshold (just African and Asian have to be re-done)
* Find similarities in really wrong predictions

In [1]:
!cd /tf/notebooks

In [2]:
!pwd

/tf/notebooks


# Import Libraries

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle
import seaborn as sns
import random
import re
from sklearn.metrics import roc_auc_score, classification_report, precision_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
import scipy.stats as st
import tensorflow as tf

from utils import create_base_race_df, create_evaluation_dataset, verify_faces

import cv2


In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
#!pip install deepface==0.0.73 --force-reinstall

In [6]:
from deepface import DeepFace

# Google FaceNet

In [ ]:
model_names = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib"]

In [ ]:
vgg_face = DeepFace.build_model("VGG-Face")

In [ ]:
facenet512 = DeepFace.build_model("Facenet512")

In [ ]:
arcface = DeepFace.build_model("ArcFace")

In [ ]:
metrics = ["cosine", "euclidean", "euclidean_l2"]

# Evaluation concept

Frame as binary classification problem and use AUROC as metric

## African

In [ ]:
# Create new validation data


# evaluation_african = create_base_race_df('African', size = 10000)
# eval_dataset_african = create_evaluation_dataset(evaluation_african)

In [ ]:
%%time
with tf.device('/gpu:0'):
    for i in range(1,6):
        data = pd.read_pickle(f'data/val_data/African/result_dict{i}.pickle')
        results = verify_faces(df = data,
                        model = vgg_face, 
                        face_count=7000)
        with open(f"results/vgg_face/African/result_dict{i}.pickle", "wb") as file:
            pickle.dump(results, file)

## Asian

In [ ]:
# Create new validation data

# evaluation_asian = create_base_race_df('Asian', size = 100000)
# eval_dataset_asian = create_evaluation_dataset(evaluation_asian)

In [ ]:
%%time
with tf.device('/gpu:0'):
    for i in range(1,6):
        data = pd.read_pickle(f'data/val_data/Asian/result_dict{i}.pickle')
        results = verify_faces(df = data,
                        model = vgg_face, 
                        face_count=7000)
        with open(f"results/vgg_face/Asian/result_dict{i}.pickle", "wb") as file:
            pickle.dump(results, file)

## Indian

In [ ]:
# create new validation data

# evaluation_indian = create_base_race_df('Indian', size = 100000)
# eval_dataset_indian = create_evaluation_dataset(evaluation_indian)

In [ ]:
%%time
with tf.device('/gpu:0'):
    for i in range(1,6):
        data = pd.read_pickle(f'data/val_data/Indian/result_dict{i}.pickle')
        results = verify_faces(df = data,
                        model = vgg_face, 
                        face_count=7000)
        with open(f"results/vgg_face/Indian/result_dict{i}.pickle", "wb") as file:
            pickle.dump(results, file)

## Caucasian

In [ ]:
# Create new validation data

# evaluation_caucasian = create_base_race_df('Caucasian', size = 100000)
# eval_dataset_caucasian = create_evaluation_dataset(evaluation_caucasian)

In [ ]:
%%time
with tf.device('/gpu:0'):
    for i in range(1,6):
        data = pd.read_pickle(f'data/val_data/Caucasian/result_dict{i}.pickle')
        results = verify_faces(df = data,
                        model = vgg_face, 
                        face_count=7000)
        with open(f"results/vgg_face/Caucasian/result_dict{i}.pickle", "wb") as file:
            pickle.dump(results, file)

In [ ]:
# PROCESSING STEPS

#from PIL import Image
#picture= Image.open(r'data/race_per_7000/African/m.05hgfh/13-FaceId-0_align.jpg') 
#picture2= Image.open(r'data/race_per_7000/African/m.05hgfh/13-FaceId-0_align.jpg')
#
#test= tf.image.rgb_to_grayscale(picture)
#DeepFace.verify(img1_path = np.array(picture),
                          #img2_path = np.array(picture2),
                          #distance_metric = metrics[0],
                        #model = facenet512,
                        #enforce_detection=False)

# Predict

In [7]:
vgg_face = DeepFace.build_model("VGG-Face")

In [8]:
tf.debugging.set_log_device_placement(True)


In [9]:
%%timeit
with tf.device('/GPU:0'):
    data = pd.read_pickle(f'data/val_data/Caucasian/result_dict1.pickle')
    results = verify_faces(df = data,
                    model = vgg_face, 
                    face_count=100)

24.7 s ± 2.64 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
with tf.device('/CPU:0'):
    data = pd.read_pickle(f'data/val_data/Caucasian/result_dict1.pickle')
    results = verify_faces(df = data,
                    model = vgg_face, 
                    face_count=100)

2022-03-17 18:42:14.938853: E tensorflow/stream_executor/cuda/cuda_driver.cc:802] failed to alloc 536870912 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory


41.9 s ± 1.75 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
df=data
model=vgg_face
performance_metrics= [roc_auc_score, accuracy_score]
face_count= 100
distance_metric='cosine'

In [ ]:
df['distance'] = np.nan
df['verified'] = np.nan

df=df.sample(frac=1)[:face_count]

for i in range(0,len(df)):

    verification_result = DeepFace.verify(img1_path = df.iloc[i].original_image,
                      img2_path = df.iloc[i].compare_image,
                      distance_metric = distance_metric,
                       model = model,
                    enforce_detection=False)
    df.distance.iloc[i] = verification_result['distance']
    df.verified.iloc[i] = verification_result['verified']

df = normalize_distances(df)
results = {}
results['data'] = df
results['max_threshold_to_verify'] = verification_result['max_threshold_to_verify']

for metric in performance_metrics:
    results[metric.__name__] = metric(df.is_same_person.astype(bool),df.verified_normalised.astype(bool))

In [ ]:
df